In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
import math

# Parameters
learning_rate = 0.0001 #0.001
training_epochs = 120 #15
batch_size = 100
display_step = 1

C:\___Anaconda\envs\forTFgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\___Anaconda\envs\forTFgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\___Anaconda\envs\forTFgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\___Anaconda\envs\forTFgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Futu

In [2]:
import csv
import os
import numpy as np

#인코딩중요
#라벨만들기
with open('./trainSet.csv', 'r', encoding='UTF-8-sig') as raw:
    lines = raw.readlines()
data = list(csv.reader(lines))
data = sorted(data, key=lambda x: x[0])

#이름빼기
_Y = [y[1] for y in data]

_Y = np.array(_Y)
_Y = _Y.astype(float) # shape(-1)
_Y = [[1-y, y] for y in _Y] # shape(-1, 2)


#데이터만들기
with open('./features4096.csv', 'r', encoding='UTF-8-sig') as raw:
    lines = raw.readlines()
data1 = list(csv.reader(lines))
data1 = sorted(data1, key=lambda x: x[0])

#데이터만들기
with open('./features_3g_16.csv', 'r', encoding='UTF-8-sig') as raw:
    lines = raw.readlines()
data2 = list(csv.reader(lines))
data2 = sorted(data2, key=lambda x: x[0])

data = zip(data1,data2)

#이름빼기
_X = [x[1:] + z[1:] for x, z in data]

_X = np.array(_X)
_X = _X.astype(float) # shape(-1, 1024)


all_set = tuple(zip(_X, _Y)) # shape(10000, 2, ~)
M_all_set = [all_set[i] for i in range(len(all_set)) if all_set[i][1][1] == 1] # 7000
N_all_set = [all_set[i] for i in range(len(all_set)) if all_set[i][1][0] == 1] # 3000
np.random.shuffle(M_all_set)
np.random.shuffle(N_all_set)

# 악성,정상 개수 맞추기
if len(M_all_set) > len(N_all_set):
    M_all_set = M_all_set[:len(N_all_set)]
else:
    N_all_set = N_all_set[:len(M_all_set)]

all_set = M_all_set + N_all_set
np.random.shuffle(all_set) # 6000

train_set = all_set[len(all_set)//5:] # 4800
test_set = all_set[:len(all_set)//5] # 1200

train_X = np.array([X for X,Y in train_set])
train_Y = np.array([Y for X,Y in train_set])
test_X = np.array([X for X,Y in test_set])
test_Y = np.array([Y for X,Y in test_set])


M_test_X = [X for X,Y in test_set if Y[1] == 1] # label의 악성이면
M_test_Y = [Y for X,Y in test_set if Y[1] == 1] # label의 악성이면

N_test_X = [X for X,Y in test_set if Y[0] == 1] # label의 정상이면
N_test_Y = [Y for X,Y in test_set if Y[0] == 1] # label의 정상이면



In [6]:
with tf.device('/gpu:0'):
    X = tf.placeholder(tf.float32, [None, 8192])
    Y = tf.placeholder(tf.float32, [None, 2])
    keep_prob = tf.placeholder(tf.float32)

    #X = tf.nn.dropout(X, keep_prob=keep_prob)

    # Layer1
    W1 = tf.get_variable('%f'%(np.random.random()), shape=[8192,1932], initializer=tf.contrib.layers.xavier_initializer())
    b1 = tf.Variable(tf.random_normal([1932]))
    L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
    L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

    # Layer2
    W2 = tf.get_variable('%f'%(np.random.random()), shape=[1932,25], initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.Variable(tf.random_normal([25]))
    L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
    L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

    # Layer3
    #W3 = tf.get_variable('%f'%(np.random.random()), shape=[456,107], initializer=tf.contrib.layers.xavier_initializer())
    #b3 = tf.Variable(tf.random_normal([107]))
    #L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
    #L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

    # Layer4
    #W4 = tf.get_variable('%f'%(np.random.random()), shape=[107,25], initializer=tf.contrib.layers.xavier_initializer())
    #b4 = tf.Variable(tf.random_normal([25]))
    #L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
    #L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

    # Layer5
    #W5 = tf.get_variable('%f'%(np.random.random()), shape=[250,100], initializer=tf.contrib.layers.xavier_initializer())
    #b5 = tf.Variable(tf.random_normal([100]))
    #L5 = tf.nn.relu(tf.matmul(L4, W5) + b5)
    #L5 = tf.nn.dropout(L5, keep_prob=keep_prob)
    
    # Layer7
    W7 = tf.get_variable('%f'%(np.random.random()), shape=[25,2], initializer=tf.contrib.layers.xavier_initializer())
    b7 = tf.Variable(tf.random_normal([2]))
    hypothesis = tf.matmul(L2, W7) + b7


    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [13]:
# Training
sess = tf.Session()
sess.run(tf.global_variables_initializer())
#sess.run(tf.initialize_all_variables())

# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(len(train_X) / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = train_X[i*100:(i+1)*100], train_Y[i*100:(i+1)*100]
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7}
        #feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        
        avg_cost += c / total_batch #뭉치갯수로 나누기
        #time.sleep(0.03)
        
    if (epoch+1)%5 == 0:
        print('Epoch : ', '%04d' % (epoch + 1), 'cost = ', '{:.9f}'.format(avg_cost))


Epoch :  0005 cost =  0.411054120
Epoch :  0010 cost =  0.322032330
Epoch :  0015 cost =  0.251946370
Epoch :  0020 cost =  0.218256515
Epoch :  0025 cost =  0.190696922
Epoch :  0030 cost =  0.152913343
Epoch :  0035 cost =  0.129262342
Epoch :  0040 cost =  0.133905821
Epoch :  0045 cost =  0.106096377
Epoch :  0050 cost =  0.102014664
Epoch :  0055 cost =  0.089971895
Epoch :  0060 cost =  0.091746907
Epoch :  0065 cost =  0.098784833
Epoch :  0070 cost =  0.094053672
Epoch :  0075 cost =  0.083136619
Epoch :  0080 cost =  0.090016966
Epoch :  0085 cost =  0.096934098
Epoch :  0090 cost =  0.084591080
Epoch :  0095 cost =  0.050647514
Epoch :  0100 cost =  0.050130016
Epoch :  0105 cost =  0.036959566
Epoch :  0110 cost =  0.053012695
Epoch :  0115 cost =  0.030532508
Epoch :  0120 cost =  0.029000689


In [14]:
# TEST
is_correct = tf.equal(tf.arg_max(hypothesis, 1), tf.arg_max(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

#feed_dict = {X: test_X, Y: test_Y, keep_prob: 1}
#print("Accuracy : ", accuracy.eval(session=sess, feed_dict=feed_dict))

#feed_dict = {X: M_test_X, Y: M_test_Y, keep_prob: 1}
#print("정탐율 : ", accuracy.eval(session=sess, feed_dict=feed_dict))

#feed_dict = {X: N_test_X, Y: N_test_Y, keep_prob: 1}
#print("오탐율 : ", 1 - accuracy.eval(session=sess, feed_dict=feed_dict))

total_batch = math.ceil(len(test_X) / batch_size)
total_acc = 0
for i in range(total_batch):
    batch_xs, batch_ys = test_X[i*batch_size:(i+1)*batch_size], test_Y[i*batch_size:(i+1)*batch_size]
    feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 1}
    one_acc = accuracy.eval(session=sess, feed_dict=feed_dict)
    total_acc += one_acc * len(batch_ys)
print("Accuracy : %0.4f"%(total_acc/len(test_Y)))


total_batch = math.ceil(len(M_test_X) / batch_size)
total_acc = 0
for i in range(total_batch):
    batch_xs, batch_ys = M_test_X[i*batch_size:(i+1)*batch_size], M_test_Y[i*batch_size:(i+1)*batch_size]
    feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 1}
    one_acc = accuracy.eval(session=sess, feed_dict=feed_dict)
    total_acc += one_acc * len(batch_ys)
print("정탐율 : %0.4f"%(total_acc/len(M_test_Y)))


total_batch = math.ceil(len(N_test_X) / batch_size)
total_acc = 0
for i in range(total_batch):
    batch_xs, batch_ys = N_test_X[i*batch_size:(i+1)*batch_size], N_test_Y[i*batch_size:(i+1)*batch_size]
    feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 1}
    one_acc = accuracy.eval(session=sess, feed_dict=feed_dict)
    total_acc += one_acc * len(batch_ys)
print("오탐율 : %0.4f"%(total_acc/len(N_test_Y)))


Accuracy : 0.9167
정탐율 : 0.9772
오탐율 : 0.8617
